#Coding Test: Train a YOLOv8 Classification Model for Car Make-Model
by Hafiz (Produced on 26 Aug 2024)

## 1. Dataset Preparation

1. **Download the Dataset**: You'll need to download the dataset from the provided link.
2. **Organize the Dataset**: After downloading, we need to extract the dataset and organize it into train and validation sets.
3. **Splitting the Dataset**: The dataset isn't split into train and validation sets, we will need to do this manually.

In [1]:
drive_link = 'https://drive.google.com/file/d/146rPq89DbjnyzevKwguwuxwkItbNhQvN/view'
test_size = 0.2

In [2]:
import gdown

def extract_file_id(drive_link):
    # Split the link based on '/d/' and '/view' to isolate the file ID
    try:
        file_id = drive_link.split('/d/')[1].split('/view')[0]
        return file_id
    except IndexError:
        return None

gdown.download(f'https://drive.google.com/uc?id={extract_file_id(drive_link)}', 'made_model.tgz', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=146rPq89DbjnyzevKwguwuxwkItbNhQvN
To: /content/made_model.tgz
100%|██████████| 88.6M/88.6M [00:04<00:00, 19.4MB/s]


'made_model.tgz'

In [3]:
import os
import tarfile

def extract_tgz_file(tgz_file, output_dir):
    # Check if the output directory exists, if not, create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the .tgz file and extract all contents
    with tarfile.open(tgz_file, "r:gz") as tar:
        tar.extractall(path=output_dir)
    print(f"Extracted {tgz_file} successfully to {output_dir}.")

tgz_file = 'made_model.tgz'  # Path to your .tgz file
output_directory = 'car_model'  # Directory where you want to extract the contents

extract_tgz_file(tgz_file, output_directory)


Extracted made_model.tgz successfully to car_model.


In [4]:
import shutil
from sklearn.model_selection import train_test_split

# Set the paths
dataset_dir = 'car_model/made_model'
output_dir = 'car_model'

train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')

# Ensure the output directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Iterate over each class folder in the dataset
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)

    if os.path.isdir(class_dir):
        images = os.listdir(class_dir)

        # Split the images into train and validation sets
        train_images, val_images = train_test_split(images, test_size=test_size, random_state=42)

        # Copy the images to the respective folders
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)

        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        for img in train_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(train_class_dir, img))

        for img in val_images:
            shutil.copy(os.path.join(class_dir, img), os.path.join(val_class_dir, img))

shutil.rmtree('car_model/made_model')

print("Dataset successfully split into train and validation sets.")

Dataset successfully split into train and validation sets.


In [5]:
import os

def count_files_in_directory(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files

print(f"Number of training images: {count_files_in_directory(train_dir)}")
print(f"Number of validation images: {count_files_in_directory(val_dir)}")

Number of training images: 960
Number of validation images: 241


## 2. Environment Setup

Install Necessary Libraries

In [6]:
!pip install ultralytics

## 3. Model Training

In [8]:
from ultralytics import YOLO

In [9]:
model = YOLO('yolov8n-cls.pt')  # load a pretrained YOLOv8n classification model
model.to('cuda')
results = model.train(data='car_model', epochs=5)  # train the model

engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=car_model, epochs=5, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynami

train: Scanning /content/car_model/train... 960 images, 0 corrupt: 100%|██████████| 960/960 [00:04<00:00, 196.61it/s]

train: WARNING ⚠️ /content/car_model/train/perodua_axia/1420224231712-18_2024-05-01T05-58-21.553089_76d6f799d1944c8089bbcc8705192188_Toyota_Proace_white_Sedan_bgp4363_61-368-258-402.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/car_model/train/perodua_axia/1420224231712-18_2024-05-02T06-09-58.721769_f66e6cb0aa1448e7a1c8e0ac71d47004_Dacia_Sandero_black_SUV_bdg9395_71-175-165-196.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/car_model/train/perodua_axia/1420224231712-18_2024-05-06T09-09-09.717824_f8330257078e4d4fa73c4c7a47f414bb_Toyota_Vios_black_SUV_vgu8471_121-248-262-277.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/car_model/train/perodua_axia/1420224231712-18_2024-05-11T13-16-52.167798_1074b95d54704292bb9d6dc9f36a637d_Hyundai_i10_silver_Sedan_wb1197_334-281-489-308.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/car_model/train/perodua_axia/1420224231712-18_2024-05-22T13-30-57.039127_55bb5fb3354845aca7aab0aab79d2d


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/car_model/val... 240 images, 0 corrupt: 100%|██████████| 240/240 [00:01<00:00, 136.98it/s]

val: WARNING ⚠️ /content/car_model/val/perodua_axia/1420224231712-18_2024-05-19T12-32-32.425905_c0365696f7c4453e865bf8a348382bac_Hyundai_i10_blue_Sedan_wb2950j_88-349-274-385.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/car_model/val/perodua_axia/1420224231712-18_2024-05-22T09-55-57.974126_3422dd94f7e1469c8b0a6c344c874564_Hyundai_i20_silver_SUV_vgh9126_158-318-334-351.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/car_model/val/perodua_axia/1423323040359-20_2024-04-04T10-22-46.034675_570b2e7220a745fda990117ca3f8e89e_Kia_Picanto_black_Sedan_vbg3655_75-322-229-361.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/car_model/val/perodua_axia/1423323040359-20_2024-04-05T12-22-32.090468_c6b4020daa064341aefeb2823f735435_None_None_None_None_wc540_683-228-801-268.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/car_model/val/perodua_axia/1423323040359-20_2024-04-07T18-17-18.459055_5246845891fd4642a43d918a743734f0_Kia_Picanto_white_Sedan_v

...

               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:02<00:00,  3.37it/s]

                   all       0.85          1



5 epochs completed in 0.023 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 3.0MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics YOLOv8.2.82 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: /content/car_model/train... found 960 images in 3 classes ✅ 
val: /content/car_model/val... found 240 images in 3 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:01<00:00,  5.72it/s]


                   all      0.867          1
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
Results saved to runs/classify/train2


## 4. Inference

In [10]:
import numpy as np
from google.colab import files
uploaded_files = files.upload()
uploaded_files = list(uploaded_files.keys())[0]

model = YOLO('/content/runs/classify/train/weights/best.pt')
results = model(uploaded_files)

# Access the probabilities object
probs = results[0].probs.data.cpu()

# Get the index of the class with the highest probability
highest_prob_index = int(np.argmax(probs))

# Get the class name with the highest confidence
class_name = results[0].names[highest_prob_index]

print(f"Car type: {class_name}")

Saving PERODUA_Axia-2019_main.jpg to PERODUA_Axia-2019_main (2).jpg

image 1/1 /content/PERODUA_Axia-2019_main (2).jpg: 224x224 perodua_axia 0.69, perodua_myvi 0.28, toyota_vios 0.02, 6.7ms
Speed: 10.4ms preprocess, 6.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Car type: perodua_axia


## 5. TensorRT (INT8 Precision) Conversion

In [ ]:
model = YOLO("/content/runs/classify/train/weights/best.pt")
model.export(
    format="engine",
    dynamic=True,
    batch=8,
    workspace=4,
    int8=True,
    data="car_model",
)

# Load the exported TensorRT INT8 model
model = YOLO("/content/runs/classify/train/weights/best.engine", task="classify")

# Run inference
result = model.predict("/content/PERODUA_Axia-2019_main.jpg")

In [13]:
# Load the exported TensorRT INT8 model
model = YOLO("/content/runs/classify/train/weights/best.engine", task="classify")

# Run inference
result = model.predict("/content/PERODUA_Axia-2019_main.jpg")

# Access the probabilities object
probs = result[0].probs.data.cpu()

# Get the index of the class with the highest probability
highest_prob_index = int(np.argmax(probs))

# Get the class name with the highest confidence
class_name = results[0].names[highest_prob_index]

print(f"Car type: {class_name}")

Loading /content/runs/classify/train/weights/best.engine for TensorRT inference...

image 1/1 /content/PERODUA_Axia-2019_main.jpg: 640x640 perodua_axia 0.49, perodua_myvi 0.26, toyota_vios 0.25, 1.5ms
Speed: 15.9ms preprocess, 1.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)
Car type: perodua_axia
